## Model Viewer Demo

This goal of this Notebook is to show that we can use the model annotation to easoly read a source position in 2 different frames.

We proceed with data mapped on individual Measure (Posistion in facty) objects in order to avoid bias possibly introduced by some host model.

This workflow validates the [Measure](https://github.com/ivoa/modelinstanceinvot-code) model (and its coordinates(https://github.com/ivoa-std/CoordinateDM)) in the extand of the the implemented classes.

### Test Case

The VOtable has been queried from Vizier (IX/57 Chandra catalogue V2):
We have 10 sources with 2 positions (ICRS and Gal), all with an ellipic error.

- We are using the annotation [syntax](https://github.com/ivoa-std/ModelInstanceInVot) that has been designed after the 2021 workshop.
- The Python code used for this notebook is being [developped](https://github.com/ivoa/modelinstanceinvot-code) to design qnd validate the processing of model annotation.


### Package Init
- Import packages
- Set the data directory
- Print to check

In [1]:
import os, sys
import xmltodict
import lxml
from astropy.io.votable import parse

base_dir = os.getcwd()
sys.path.append(os.path.join(base_dir, "python") )
data_dir = os.path.join(base_dir, "python", "tests", "data", "annotated_data") 

from utils.xml_utils import XmlUtils
from utils.dict_utils import DictUtils
from client.xml_interpreter.model_viewer import ModelViewer

print(sys.path)
print(base_dir)
print(data_dir)


## VOTable parsing
The VOTable is parsed by Astropy as usual

In [3]:
votable = parse(os.path.join(data_dir, "vizier_csc2_gal.annot.xml"))

## Building the Model Viewer
The model viewer wrapps the VOTable data iterator to provide model views of the current data raw
The model viewer supports 3 serializations:
- XML
- JSON 
- STC components (not used here)

In the future, the resource selection should be handled by the either Astropy or PyVO, but meanwhile this is not implemented, the job is done by hand. 
We assume that the 1st resource is annotated.

In [5]:
for resource in votable.resources:
    mviewer = ModelViewer(resource, votable_path=os.path.join(data_dir, "vizier_csc2_gal.annot.xml"))
    break

## Connecting the Data table we Want to Browse
In this example we just have one table that is annotated. 
- We assume we will work on data of this table
- We do not look for complex objects that could be located in some other tables or in the GLOBALS block 

In [7]:
mviewer.connect_table('Results')


## Print Numpy Row

As an evidence we arwe really working with the astropy parser.

In [8]:
print(mviewer.get_next_row() )

## XML model View

We use a (not so good) local utility to print ou the the XML model view of the current row.
Notice that the attributes have a @colindex attributes that is no part of the annotation syntax. 
This is for a debug purpose.

In [14]:
XmlUtils.pretty_print(mviewer.get_model_view(resolve_ref=True))


### JSON Model View 

We use a (pretty good) local utility to print ou the the JSON model view of the current row. 
Notice that the JSON should be cleanup. This is not done for a debug purpose.

In [11]:
DictUtils.print_pretty_json(mviewer.get_json_model_view(resolve_ref=False))

## Retrieving the Space Frames

This simple functions returns the list of all space frames present in the annotation block by using a model dmtyp selector.

In [13]:
DictUtils.print_pretty_json(mviewer.get_json_model_component_by_type("coords:SpaceFrame"))